# Initialization vs Isometry in Qiskit

According to `201 Tips and tricks.ipynb` there are exist two approaches to prepare the state: one was proposed by Shende, Bullock and Markov in ["Synthesis of Quantum Logic Circuits"](https://arxiv.org/abs/quant-ph/0406176) (2004) and another — by Iten et al. in ["Quantum Circuits for Isometries"](https://arxiv.org/abs/1501.06911) (2020). For more details please refer to correspondent notebook and original papers.

This notebook concerns comparison of these two methods in terms of how close obtained states are to the desired vector. For someone who knows quantum programming inside out, the answer might appear trivial, but not for me...

### Prerequisites

In [1]:
import math
import numpy as np
from qiskit import QuantumCircuit, BasicAer, execute, transpile
from scipy.stats import wasserstein_distance as ws

In [2]:
class Experiment:
    
    def __init__(self, vector_size=16):
        message = "Vector size should be power of 2"
        assert (vector_size & (vector_size-1) == 0) and vector_size != 0, message
        
        self.vector_size = vector_size
        self.n = math.ceil(math.log(self.vector_size, 2))
        
    def run(self, shots=10000):
        data = self.__get_random_vector()
        init_dict = self.__apply_initalize(data, shots)
        
        result = dict(
            init = init_dict,
        )
        return result
    
    def __apply_initalize(self, data, shots):
        n = self.n
        qc = QuantumCircuit(n, n)
        qc.initialize(data)
        qc.measure(range(n), range(n))
        
        counts = execute(qc, BasicAer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
        # TODO: sometimes number of elements there is less than vector_size (pad)
        dist = np.array([b for a,b in sorted(list(counts.items()), key = lambda x: x[0])]) 
        dist = dist / np.linalg.norm(dist)
        
        metrics = dict(
            mse = np.square(data - dist).mean(),
            ws = ws(data, dist)
        )
        return metrics
        
    
    def __get_random_vector(self):
        a = np.random.rand(self.vector_size)
        a = a / np.linalg.norm(a)
        return a

In [3]:
experiment = Experiment(32)

In [4]:
experiment.run()

{'init': {'mse': 0.0016230030811164708, 'ws': 0.03584424375870574}}